In [13]:
! pip install transformers
! pip install fpdf
! pip install torch
! pip install pathlib
! pip install moviepy.editor
! pip install ffmpeg-python
! pip install proglog
! pip install PyPDF2

ERROR: Could not find a version that satisfies the requirement moviepy.editor (from versions: none)
ERROR: No matching distribution found for moviepy.editor
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.5 MB/s eta 0:00:00


In [28]:
from pathlib import Path
import os
from moviepy.editor import *
from proglog import ProgressBarLogger

%cd /content/drive/MyDrive/Colab

class MyBarLogger(ProgressBarLogger):

    def bars_callback(self, bar, attr, value,old_value=None):
        percentage = (value / self.bars[bar]['total']) * 100
        #Print the percentage of the convertion
        print(percentage)

logger = MyBarLogger()

#pathVideo is a list of the Path of the videos
def useConverter(pathVideo) :

    pathList = []

    for path in pathVideo:
      video = VideoFileClip(str(path))

      if not os.path.exists("Mp3"):
        os.makedirs("Mp3")
      pathMp3 = Path("Mp3") / (str(path).replace(".mp4", ".mp3"))

      if not pathMp3.exists():
        pathList = pathList + [pathMp3]

        #Converter
        video.audio.write_audiofile(pathMp3, logger=logger)

    #Call whisper for the transcription
    #Whisper.useWhisper(pathList)

useConverter([ Path("Lezione1.mp4"), Path("Lezione2.mp4"), Path("Lezione3.mp4")] )

/content/drive/MyDrive/Colab


In [8]:
from transformers import pipeline
import torch

model = pipeline(
    task="automatic-speech-recognition",
    model="openai/whisper-large",
    device="cuda:0",
    torch_dtype=torch.float16,
)

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.85k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [27]:
#Converter from mp4 to mp3
#import Whisper
from pathlib import Path
from moviepy.editor import *
from fpdf import FPDF
import os
from pathlib import Path
from transformers import pipeline
import torch
import PyPDF2

%cd /content/drive/MyDrive/Colab

#pathPdf is a list of the Path of the pdfs
#output_path is the output of the merged pdfs
#merge the pdfs
def merge_pdfs(pathPdf, output_path):

    if not os.path.exists("Merged"):
       os.makedirs("Merged")
    pdf_writer = PyPDF2.PdfWriter ()

    for pdf_path in pathPdf:
        pdf_reader = PyPDF2.PdfReader(str(pdf_path))
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            pdf_writer.add_page(page)

    with open(output_path, 'wb') as out_file:
        pdf_writer.write(out_file)

#pathVideo is a list of the Path of the mp3s
def useConverter(pathMp3) :

    listPdf = []

    for path in pathMp3:

      #Converter
      #Call whisper for the transcription
      if not os.path.exists("Pdf"):
        os.makedirs("Pdf")

      pathPdf = Path("Pdf") / (str(path).replace(".mp3", ".pdf").replace("Mp3/", ""))
      listPdf = listPdf + [pathPdf]
      #model = tiny, base, small, medium, large
      if not pathPdf.exists():

        result = model(str(path))
        transcribed_text = result["text"]
        print(transcribed_text)

        tt= transcribed_text.encode('latin-1', 'replace').decode('latin-1')

        #save the transcribe in a file pdf
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        pdf.multi_cell(0, 10, tt)

        pdf.output(pathPdf)

    merge_pdfs(listPdf, "merge.pdf")

#useConverter([Path("Mp3/Lezione1.mp3"), Path("Mp3/Lezione2.mp3"), Path("Mp3/Lezione3.mp3")])
#merge_pdfs([Path("Pdf/Lezione1.pdf"), Path("Pdf/Lezione2.pdf"), Path("Pdf/Lezione3.pdf")], Path("Merged/merge.pdf"))

/content/drive/MyDrive/Colab


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import gc

gc.collect()
torch.cuda.empty_cache()

NameError: name 'torch' is not defined

In [ ]:
import os
%cd /content/drive/MyDrive/Colab/Mp3


result = model("Voce-024.mp3")
print(result)

/content/drive/MyDrive/Colab/Mp3


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


{'text': " L'idea è completamente abbandonata per un motivo in realtà molto più base, nel senso che io ho sempre compilato PX4 per ARM e STM32 il firmware, adesso che lo compilo per il simulatore sembra quasi che non supportino più certi tipi di veicoli. Con la documentazione, la documentazione dice di fare delle cose e di compilare per certi target che non esistono più. Quindi non è che non riusciamo a farlo, soltanto che non ci sono più come dire i target per il simulatore. Quando io compilo però per arma, quindi con il compilatore di armi, il target esiste. Quindi nel makefile loro probabilmente hanno tolto qualcosa o non supportano più certi tipi di simulazione perché non... A me sembra strano questa cosa. Scusami. PX4 noi stiamo parlando del simulatore, Cazzevolo, giusto per Cazzevolo. Esatto, esatto. Ti dico questo perché il PIX4 in realtà non è neanche di per sé un drone vero e proprio, il PIX4 è un autopilot, che può essere poi montato a bordo di diversi hardware. Non vorrei ch

In [ ]:
%cd /content/drive/MyDrive/Colab/Mp3/
!ffmpeg -i RiunioneDroniLezione1.mp3 -f segment -segment_time 300 -c copy out%03d.mp3

/content/drive/MyDrive/Colab/Mp3
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable

In [ ]:
%cd /content/drive/MyDrive/Colab/
!ffmpeg -i "Lezione1.mp4" -q:a 0 -map a Mp3/output.mp3 -y

/content/drive/MyDrive/Colab
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-lib

In [ ]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import transformers
import torch

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-7B-Instruct",
    torch_dtype=torch.float16,
    device_map="cuda",
    quantization_config=quantization_config,
)

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")



In [ ]:
messages = [
    {"role": "user", "content": """ Ciao, riesci a crearmi una dispensa a partire da questo: Poi vi lascerò questi materiali nell'area diciamo registrazione di Teams questi oppure quelli degli anni scorsi se dovessero venire meglio, se dovessero essere migliori diciamo. Seconda cosa, allora voi mi dite che tendenzialmente sarete sempre in aula a Udine, chi è a Udine giusto? Ok quindi manteniamo questo canale e io come vi accennavo nel messaggio che vi ho mandato stamattina principalmente sarò qui a Pordenone in presenza, può capitare qualche volta che sarò lì a Udine, in particolare ad esempio domani potrebbe succedere perché c'è un consiglio di Udine quindi devo essere lì già per il consiglio, è probabile che mi fermi per la lezione ma altrimenti se non vi do notizia sono qui e in ogni caso diciamo il canale Teams è sempre acceso quindi chi vuole partecipare via Teams. Ecco una cosa che non era prevista era la diretta Teams perché ci dicono che in generale bisognerebbe registrare ma non ma non avere il canale Teams in streaming diciamo nel temporaneo. Io gli anni scorsi l'ho tenuto acceso e chiedo adesso a chi è in Teams, Viola forse tu mi sai rispondere, tu seguirai sempre via Teams e non in aula giusto? In realtà io lavoro e faccio i turni quindi alcune volte forse non riuscirò neanche a seguire via Teams però a volte potrebbe anche essere che invece riesco anche a venire in presenza quindi purtroppo non lo so con precisione perché ogni settimana cambiano i turni quindi. Va benissimo ma allora per fronte a questa situazione io direi lascerò sempre la diretta anche accesa tanto insomma nella registrazione ci sarà anche uno streaming in diretta facciamo così. Allora questo devo condividere, Viola tu non vedi le mie slide mentre a Udine sì scusate all'inizio sono un po' frastornato ma dopo dovrei capire come funziona. Se qualche volta vado avanti senza avervi condiviso magari avvisatemi. Ecco adesso dovresti vedere anche tu se non sbaglio. Sì grazie. Ecco questa è la presentazione del corso, scusate che vi presento il powerpoint anziché le slides. Questo è un corso da sei crediti come saprete, è un corso tenuto da me Carlo Brioli e dal secondo docente si chiama Daniele Salvati che entrerà a far lezione al mio posto nelle ultime settimane del corso. Quindi io vi accompagnerò fino a grossomodo i primi due terzi del corso e poi ci avvicineremo con il professor Salvati. Nella stessa slide che voi trovate nel Moodle, adesso vi mostro le pagine del Moodle che voi trovate nella vostra collezione corsi. Il corso ha un codice che quest'anno è 4663. Ok, potete arrivarci direttamente. In queste pagine ogni volta che facciamo lezione o un giorno prima o qualche ora prima carico le slide della giornata e tutti i materiali che possono essere utili per quella giornata oppure per le volte future. Nel Moodle del corso trovate anche il link per accedere al canale Teams, obiettivi e linee guida che sostanzialmente riflettono il syllabus, dovrebbe essere aggiornato a quello che vedremo quest'anno e poi ogni sezione ogni giorno fa riferimento alla propria sezione. Oggi faremo l'introduzione e qualcosa di Mobile Robotics, inizieremo a inquadrare un po' gli aspetti che seguiremo nel corso. Come è organizzato dunque Vi dicevo è un corso da 6 crediti quindi sono 42, sono 12 lezioni, sono 21 lezioni scusate, che si tengono il martedì quest'ora e domani il mercoledì 12 e 30, 14 e 30, sono lezioni principalmente frontali e ogni tanto però organizzeremo delle lezioni più sperimentali, più laboratoriali, o meglio organizzeremo, qui dovete anche dirmi voi quanto la cosa è interessante dal vostro punto di vista per la partecipazione, cioè le lezioni laboratoriali in alcuni casi potrebbero essere coinvolgere i dispositivi fisici, quindi l'utilizzo di dispositivi robotici in aula e quindi potrebbe essere un'occasione per una partecipazione diretta. Non sempre è possibile, questo naturalmente è ovvio, allora nel caso in cui ci sia una possibilità di partecipare in maniera diretta a Udine, a limite qualche occasione delle lezioni laboratoriali possiamo organizzare le cose in modo da essere lì in aula a Udine, ve lo saprò dire però più avanti perché qui a Pordenone noi abbiamo il laboratorio e una serie di attrezzature che non sempre è facile spostare verso Udine, quindi a questo punto cercheremo di organizzarci in quel senso più avanti, altrimenti faremo una proiezione, uno streaming anche dell'esperienza laboratoriale da qui e nell'aula di Pordenone. Le lezioni cominciano oggi e vanno avanti grosso modo fino a maggio, probabilmente per fine maggio riusciamo a chiudere, adesso dipenderà anche dal docente che mi seguirà, però insomma io verso fine maggio considero prima di giugno al massimo di chiudere. Materiale didattico, anche qui come vi accennavo prima dovrebbe essere tutto reperibile sul sito e-learning, i miei libri di test e testi di riferimento invece sono in alcuni casi reperibili online e vi segnalo magari quali di questi testi quando li useremo, sono reperibili senza difficoltà, in altri casi sono reperibili in biblioteca. Io vi ho elencato una serie qui di testi di riferimento e testi di approfondimento, di testi di riferimento in particolare il primo, il Dudek-Jenkin, Computational Principles of Mobile Robotics, è un testo che seguiamo soprattutto all'inizio, quindi troverete parecchi materiali ispirati a questo testo, è un testo relativamente semplice come livello tecnico e con un giusto livello di dettaglio quando serve e naturalmente non seguiremo tutto il contenuto ma soltanto alcune sezioni, quindi anche qui usate le slide per per orientarvi anche nei contenuti dei vari testi da seguire. Il secondo testo, il Sigward Norbach-Skaramuzza, Introduction to Autonomous Mobile Robots, invece è un testo abbastanza classico del 2011 e se volete è un po' più approfondito, tratta gli argomenti in maniera più estesa rispetto al Computational Principles di prima e per cui io lo consiglierei sicuramente per chi vuole un po' approfondire certi argomenti oppure per chi vuole ampliare il panorama degli argomenti trattati con qualcos'altro, ecco non limitandosi soltanto quello che vedremo noi. Il Kagan Autonomous Mobile Robots è un testo della Wiley molto recente, uscito nel 2020 qualche anno fa, è un testo che è organizzato in capitoli curati da differenti autori, è una classica raccolta di Autored Contributions, di contributi editi dagli autori diversi. Kagan coordina l'edizione e da questo punto di vista direi che forse meno indicato come libro di testo, non ha una sua organicità, però lo usiamo per alcuni capitoli che sono ben fatti, quindi anche lì vi segnalerò, o trovate nelle slide in basso a sinistra, quando mi riferisco a certi testi in particolare trovate il riferimento. Questi tre coprono grossomodo gli aspetti teorici, sono testi principalmente teorici. Il Quigley che trovate invece in fondo, testo del 2015 della O'Reilly, è un testo un po' più pratico, il Programming Robots with ROS, e anche perché fa riferimento a un ambiente di programmazione di sistemi robotici, Robotic Operating System, che vedremo verso la metà, verso il secondo terzo del corso, e fa riferimento in particolare a problemi specifici, a esempi pratici di programmazione di algoritmi attraverso uno dei linguaggi di programmazione che useremo nel corso, il Python, e quindi vi guida verso degli aspetti più pratici. Lo incontreremo più avanti, quindi avrò modo poi di segnalarvi altri aspetti. Ho aggiunto anche un altro paio di testi di approfondimento, il Wyatt Newman, Human, scusate, Sistematic Approach to Learning Robot Programming with ROS, anche questo è un testo pratico, che affronta la programmazione dal punto di vista pratico. Questo è un testo del 2017, e contiene una serie abbastanza ampia di esempi, quindi di problemi risolti attraverso strategie di programmazione e algoritmi scritti, in particolare però, e questa è una piccola nota che volevo segnalarvi, scritti in linguaggio C, non in Python. Questo è uno dei motivi per cui io l'ho lasciato tra i testi di approfondimento e non tra i testi principali, perché, come ho accennato prima, abbiamo dovuto fare una scelta dal punto di vista dei linguaggi usati, altrimenti diventava un po' troppo dispersivo da quel punto di vista, quindi abbiamo deciso di orientarci verso il linguaggio Python, anziché il linguaggio C, questo perché so che, diciamo, voi lo incontrate in vari altri corsi, sia ad STM, sia a CNT, sia a Informatica, quindi in grosso modo dovreste avere già una certa dimestichezza con il linguaggio. Scusate, mi è venuto in mente, non vi ho chiesto, io di solito a questo punto chiedo voi a che corsi di studio fate riferimento, a che corsi afferite e anche che esperienze avete da questo punto di vista, perché mi serve un po' per organizzare i materiali, per organizzare anche il taglio della discussione tante volte. Siete di CMT o di Informatica o di Ingegneria? Qualche volta mi è capitato anche qualcuno da Ingegneria. Sei di Informatica? Ok, ok, scusami, Informatica? Terzo anno? Secondo anno? Magistrale, quindi secondo anno. Ok, Informatica Magistrale, scusami, volevo chiedere a te appunto se avete corsi in cui cominciate a lavorare in Python, in linguaggio di programmazione Python, quindi non c'è problema. Ecco, qualche cosa vedremo anche in Matlab a volte, ma molto poco e soltanto per poi eventualmente ricondurci al Python, ecco, quindi da quel punto di vista. Comunque Matlab l'avete anche quello incontrato, se non sbaglio, in Informatica. Matlab mai? Ok, benissimo, è un'informazione utile e vi servirà per evitare di allora insistere su quell'aspetto lì. E invece, scusa, chi è in connessione Teams? Stesso discorso? Siete principalmente informati? Io vengo da CMT, primo anno. Ok, primo anno. Io ho fatto il CTM e non abbiamo usato mai Python, però abbiamo fatto qualcosa con Matlab, quindi il contrario. Esattamente il contrario, questo mi conforta. Allora, però credo che se tu sei iscritta al primo anno di CMT, lo farai molto presto Python, credo. Lo vedrai molto presto con qualcuno, credo Micheloni. Forse, sono sempre la mia scelta, però. Ho capito. Io ho scelto l'indirizzo quello, oddio, non mi ricordo, per sistemi multimediali. Sistemi multimediali, dovresti probabilmente incontrarlo, se non con Micheloni, perché voi fate alcuni esami sia di Machine Intelligence e lì sicuramente lo incontrerai, oppure altri esami di visione, computer vision. Sì, sono la mia scelta, non sono obbligatori. Ho capito. Allora, noi ti dico, faremo, cominceremo a lavorare con quel linguaggio verso il secondo o terzo percorso, quindi significa fra 4-5 settimane almeno. Magari mi puoi indicare qualche cosa per partire. Esatto, io ti consiglierei, se vuoi seguire poi la parte pratica in maniera diciamo approfondita, ti consiglierei, ti potrei consigliare qualche testo di riferimento e qualcosa allora che ti aiuta così nei primi passi principalmente. Scusami, intanto, forse ancora più importante da questo punto di vista è in realtà avere degli skill di programmazione robusti. Che tipo di linguaggio hai imparato? STM, JavaScript immagino? Sì, HTML, CSS, JavaScript e poi abbiamo fatto quell'esame di programmazione in videogiochi con l'UA. Quindi le basi della programmazione, diciamo, ne hai abbastanza, quello non c'è problema, quindi non dovresti avere grossa difficoltà a passare ad un nuovo linguaggio. Considera anche che in realtà il passaggio al linguaggio Python, come poi vedremo, questa è una cosa che interesserà a tutti, ma diciamo viene usato le peculiarità del linguaggio Python rispetto ad altri linguaggi che vengono usate in realtà in modo limitato, perché come vedremo gran parte dell'infrastruttura dell'ambiente di programmazione ROS che useremo, diciamo, provvede a rendere disponibili tutta una serie di meccanismi che sarebbero complicati da gestire in un linguaggio come Python. A noi rimane la componente più semplice, cioè quella di scrivere alcune funzioni in dei template ben strutturati, quindi in realtà non ci sarà tanto da, diciamo, di nuovo da imparare dal punto di vista della sintassi, servirà naturalmente saper avere a che fare con alcuni tipi di dati tipici del Python, per esempio gli array, le matrici e alcune strutture di controllo, insomma la sintassi tipica delle strutture di controllo. Sarà invece importante imparare a gestire, per chi non l'ha mai visto, imparare a gestire questa peculiarità del Python che indenta, cioè usa l'indentazione al posto delle parentesi, quindi a differenza di altri linguaggi è importante imparare a indentare correttamente, altrimenti perché l'indentazione diventa nel caso di Python diventa un elemento sintattico del linguaggio. Quindi ecco forse è l'unica cosa da segnalare subito. Va bene vi indicherò allora domani, cerco di darvi un'anticipazione sui materiali che potreste cominciare a guardare se volete così acquisire qualche elemento di Python, chi non ce l'ha. Ecco testi, altri materiali saranno disponibili, in particolare io qui non ho indicato i testi di riferimento che riguardano la terza parte del corso, cioè la parte che curerà il professor Salvati, questo perché è una parte molto specifica su una particolare classe di sensori, in particolare sensori acustici, questo in modo da completare il quadro che voi alla fine del corso di studi di CMT e di informatica avete rispetto alla percezione in ambito informatico. Voi avete visto molto bene tutto quello che riguarda computer vision e probabilmente a questo punto, se non per qualcosa vista in STM, chi l'ha vista, vi manca la componente di computer audition, cosiddetta, se volete, una controparte di percezione uditiva che fa un po' impari con la percezione visiva, oggi è nota nell'ambito dell'informatica e del multimedia come, scusate, della computer perception, computer vision allargata a, diciamo, la computer perception in generale, è chiamata computer audition o con altri termini, ma insomma sono poco importanti, per noi è importante che imparare a, diciamo, capire come la scena acustica può essere percepita da un sistema di calcolo, da un sistema informatico, come i segnali, che natura hanno i segnali acustici e come questi vengono elaborati, visto che l'elaborazione del segnale acustico è leggermente diversa da quella del segnale visivo e, diciamo, da questo da questo punto in poi poi vedere quali tipi di algoritmi e quali tipi di logiche di analisi dei segnali oggi si conoscono per ottenere delle informazioni di alto livello, così come nella computer vision, scusate qui sono passato a questo punto all'altra slide, cioè come con la computer vision siete stati abituati, avete imparato a analizzare l'immagine o il flusso di immagini, cioè il video, fino ad ottenere delle rappresentazioni di alto livello dell'informazione visiva, così nell'audio possiamo ragionare livelli dal basso verso l'alto, partendo dal segnale per poi arrivare a delle rappresentazioni di caratteristiche timbriche o spaziali, fino ad arrivare alla comprensione della cosiddetta scena acustica, cioè a riuscire a capire se nella scena acustica è presente una sorgente di interesse, a seguirla, a decifrarla, decodificarla, a capire cosa fa eccetera, questa è un po' la parte che curerà il professor Salvatici, ritorno nella slide successiva vedo che qualcosina ho aggiunto anche lì. Tornando invece agli obiettivi più generali, l'obiettivo del corso invece è quello di darvi le basi teoriche e pratiche per avere a che fare con le tecnologie tipiche della robotica mobile e dove per robotica mobile vedremo tra un attimo intendiamo diciamo alcune classi particolari di robot mobili, non guarderemo a tutto lo spettro possibile dei dispositivi così detti mobili, così annoverati fra i robot mobili, vi farò vedere anche perché diciamo qual sarà la classe che guarderemo più da vicino, ma insomma per quelle particolari classi cercheremo di capire aspetti teorici, cercheremo di acquisire degli strumenti informatici per realizzare alcune, per implementare alcune algoritmi e alcune diciamo casi di studio interessanti, non saranno tanti ma saranno significativi secondo noi e naturalmente questo cercando di acquisire gli elementi necessari sia per quanto riguarda gli aspetti motori dei dispositivi, sia per quanto riguarda gli aspetti sensoristici, cioè sia per quanto riguarda ciò che serve a far muovere una piattaforma mobile, sia per quanto riguarda gli aspetti che servono a percepire l'ambiente in cui la piattaforma mobile si muove. Non ultimo e molto importante cercheremo di acquisire alcuni aspetti legati all'autonomia, cioè come è possibile dotare una piattaforma mobile dotata quindi di sensori e di attuatori per il movimento eccetera, come è possibile dotarla di logiche di valutazione dei dati provenienti dai sensori, logiche di decisione per capire come usare quei dati per prendere delle decisioni tipicamente rispetto al movimento dei propri attuatori in modo da poter ad esempio svolgere dei compiti, semplici compiti, muoversi all'interno di un ambiente in cui ci sono degli ostacoli, interagire con alcuni oggetti, oggetti statici, oggetti in movimento e così via. Lo faremo come vedremo sia dal punto di vista simulato proprio per semplificare l'approccio in un primo momento cercheremo di muoverci in un ambiente simulato e poi anche con alcuni esempi pratici, alcuni esempi che coinvolgono dispositivi fisici. Gli argomenti trattati intanto vedremo una prima panoramica guardando alle diverse classi di agenti aerodotici coinvolti, quindi dei loro aspetti funzionali, costruttivi, le loro capacità, le problematiche che investono il controllo del movimento di questi dispositivi. Guarderemo alcuni aspetti del controllo di dispositivi singoli ma anche di dispositivi organizzati in cluster, quindi come si dice, o in swarm, cioè in gruppi di unità che devono coordinarsi o fra loro o coordinarsi con un'unità di controllo centrale. Vedremo le diverse tipologie di sensori, rivedremo una panoramica, vedremo comunque un piccolo riepilogo di quanto riguarda, di quello che riguarda i sensori ottici e delle principali tecniche di analisi del flusso ottico, soprattutto per segnalarvi, per darvi delle indicazioni su quali sono gli aspetti da riprendere se volete, da rinfrescare, perché sono molto utilizzati in robotica. Faremo lo stesso, come vi accennavo, per quanto riguarda l'audio e poi dal punto di vista pratico impareremo a conoscere alcuni ambienti e linguaggi di programmazione che ci aiuteranno a implementare gli schemi di controllo, gli schemi di pianificazione, gli schemi di logic dell'autonomia che abbiamo visto nella prima parte, che avremo visto nella prima parte. E infine spero di presentarvi alcuni casi reali, ve li presenterò senz'altro, perché abbiamo dei materiali sviluppati in questi anni con i vostri colleghi, sicuramente dal punto di vista dell'ambiente simulato. Quindi voi sarete in grado di partire da alcuni esempi pratici sviluppati in passato, alcuni di questi esempi vi proporrò di poi svilupparli invece secondo altre direzioni, quindi alcuni spunti che vi darò e in questo senso costituiranno un po' gli aspetti di verifica pratica durante il corso e questo lo faremo quando invece cercherò di presentarvi anche alcuni aspetti, alcuni esempi reali. Allora in questo caso vi farò vedere alcune classi di sensori, alcune classi di robot piattaforme mobili, allora in quel caso specifico approfondiremo le loro caratteristiche, le loro capacità e le loro applicazioni con dei piccoli esempi pratici. La struttura del corso, la struttura come vi accennavo prima, il corso è organizzato in tre parti. La prima parte in cui vedremo le basi della robotica mobile e andremo a guardare diciamo la classificazione dei dispositivi, i tipi di movimento, come per poter studiare il movimento è necessario acquisire alcuni elementi di cinematica, di dinamica dei diversi dispositivi, ne vedremo qualcuno un po' nel dettaglio, qualcuno un po' nel dettaglio, vedremo alcuni aspetti di controllo, vedremo che il controllo serve a delle funzioni un po' più evolute come il planning, cioè la pianificazione del movimento o senza arrivare già al planning che forse è un po' più evoluto, al ciò che si chiama, scusate mi sfugge il termine, la navigazione, ok? La navigazione come vedremo è uno degli aspetti centrali dello studio delle piattaforme mobile che ha bisogno per poter diciamo essere realizzata in maniera compiuta da una piattaforma, ha bisogno degli elementi di controllo degli attuatori dei dispositivi e poi vedremo il sensing e alcuni aspetti di collective swarm, cioè di cluster di droni. Nella seconda parte vedremo gli aspetti pratici, quindi cercheremo di mettere un po' in pratica quello che abbiamo visto, però nella prima fase studieremo questo ambiente che si chiama Robotic Operating System. Qualcuno di voi ne ha già sentito mai parlare? In altri corsi non credo, ma per altri motivi no, vero? In effetti non è molto noto, probabilmente questa è la prima volta che lo vedrete, ma poi avrete nella vostra carriera professionale probabilmente avrete occasione di incontrarlo in altri contesti, perché è una piattaforma in effetti che si incontra non solo nella robotica, ma ormai in tanti contesti diversi. Non vi faccio anticipazioni particolari, ma in questa seconda parte impareremo l'architettura di questo ambiente di simulazione e impareremo a programmare alcune strategie del controllo delle unità robotiche attraverso il linguaggio di programmazione Python, come vi avevo accennato. Per fare questo ci interfacceremo con un ambiente di simulazione che si chiama Gazebo. Gazebo è un ambiente di simulazione fisica in tre dimensioni, quindi è un ambiente grafico molto interessante e al pari pesante dal punto di vista computazionale, perché permette di gestire le simulazioni dal punto di vista fisico, considerando gli aspetti di simulazione della fisica che funziona sotto, nel layer sottostante, a ciò che si vede dal punto di vista dell'animazione grafica, almeno lì dove è necessario. Poi è possibile anche gestire degli oggetti il cui movimento non è governato dalla fisica sottostante, ma è quello un altro discorso, impareremo a conoscere questi aspetti. La cosa che è interessante per noi appunto è che attraverso questo mondo simulato potremo mettere in pratica molte delle cose delle cose viste in maniera molto realistica e molto verosimile. ROS offre in particolare la possibilità di sostituire poi i suoi modelli simulati con modelli reali, quindi se poi avremo occasione di sostituire degli oggetti simulati con degli oggetti reali vi farò vedere appunto cosa succede in quel caso. Qui vi accenno anche al fatto che vedremo in particolare alcune categorie di mobile drones, in particolare affronteremo tra le tante diciamo forme di robot mobili che penso vedremo tra un attimo, ma soprattutto approfondiremo i robot dotati di ruota, quindi i cosiddetti wheeled drones e droni aerei, cioè i cosiddetti UAV per chi l'avessi già sentito nominare, cioè Unmanned Aerial Vehicles ad esempio. In particolare guarderemo una classe molto specifica e cioè la classe di droni ad elica perché hanno delle caratteristiche che ci interessano particolarmente in questo periodo, in questa fase storica diciamo l'evoluzione della robotica, hanno delle particolari caratteristiche che li rendono maneggevoli e permettono di essere manovrati anche in ambienti chiusi in molti casi, quindi potremo fare degli esperimenti pratici se le condizioni ce lo consentiranno. La terza parte infine è quella che riguarda sensori acustici in particolare, approfondimenti da parte con il professor Salvati e in questo caso vedrete un aspetto molto interessante dell'ambito della signal processing acustico, in particolare quello degli array, cioè dell'elaborazione dell'array processing, cioè l'elaborazione del segnale multicanale. Gli array sono strutture di sensori, strutture fisiche che servono a configurare collezioni di sensori secondo certe disposizioni, secondo certe geometrie e a certe geometrie corrispondono particolari caratteristiche degli array acustici e imparerete appunto con il professor Salvati a capire tutte le varie fasi che sono necessarie per il trattamento del segnale multicanale, dagli aspetti di acquisizione, quindi come vanno gestiti i sensori dal punto di vista hardware e software, come vanno gestiti i segnali ad n dimensioni, segnali acustici, quali sono gli algoritmi principali che si possono studiare su questi segnali e che applicazioni pratiche hanno, quindi imparerete ad apprezzare questo è un aspetto abbastanza diciamo poco trattato generalmente nei corsi universitari perché è un aspetto avanzato ma sicuramente è uno degli aspetti oggi che sta diciamo che sta guadagnandosi un grande interesse da parte della robotica perché appunto è stato un po' trascurato fino ad oggi invece adesso si comincia a guardare con più attenzione a questo aspetto, quindi è un'ottima occasione per approfondire. Prerequisiti in realtà non tantissimi se non gli elementi essenziali di matematica, trigonometria di base, qualcosa di analisi matematica, quindi almeno sapere cosa sono derivate integrali, come gestire le funzioni, qualcosina di trasformate, trasformate di furie in particolare perché nell'analisi video, nell'analisi audio sono naturalmente implicati questi strumenti di matematica avanzata. Niente che non sarà naturalmente comunque richiamato e così rivisto insieme dove serve. Poi aspetti di algebra lineare, naturalmente tutti gli argomenti che riguardano signal processing hanno sempre molto a che fare con l'algebra matriciale principalmente, quindi vuoi sapere che le immagini sono matrici, segnali, segnali acustici sono array, sono vettori e così via, quindi ci sarà da lavorare con questi elementi. Probabilità statistica, alcuni aspetti faremo riferimento spesso ad esempio medie, varianze, matrici di correlazione nel caratterizzare ad esempio i segnali o nel caratterizzare alcuni modelli degli ambienti in cui ci troviamo oppure dei dispositivi che andiamo a trattare, quindi probabilità statistica almeno gli elementi fondamentali. E sono ospicati skill pratici, riconoscenze, si riferite ai sistemi operativi principalmente quando lavoriamo, quando cominceremo a lavorare in ROS avremo bisogno di operare in ambiente Unix-like, quindi Linux tipicamente, quindi è ospicabile che vi sappiate un po' di stricare, sappiate un po' muovere all'interno di shell di comando, file system, principali comandi del sistema operativo, Linux e Windows, insomma questi sono gli aspetti da segnalare e poi Python come vi ho già accennato un attimo fa. Modalità d'esauto, fermatevi se mi dimentico qualcosa oppure se avete domande, siamo pochi quindi possiamo tranquillamente gestire le cose in maniera interattiva. Questo esaurisce un po' la struttura del corso in termini di contenuti. Per quanto riguarda l'esame, l'esame finale è un esame scritto per metà, cioè noi abbiamo una sorta di test scritto che verte un po' su tutto quello che abbiamo trattato durante il corso in maniera non tanto approfondita, diciamo ci sono aspetti teorici anche qualche aspetto pratico cioè a volte vi si chiede di risolvere qualche piccolo problema dal punto di vista numerico, qualche volta vi si chiede di dare una descrizione di risposta aperta, sono una decina di punti e questo serve principalmente per poi accedere alla seconda parte dell'esame che invece è la discussione di un progetto pratico, discussione, dimostrazione, dipende da che tipo di progetto poi avremo concordato. Questo è un progetto concordato con me o con il professor Salvati che può essere diciamo sviluppato o in ambiente simulato, quindi nell'ambiente che avremo imparato a conoscere durante il corso, oppure se avete interesse o se avete la possibilità di accedere ad un dispositivo robotico vostro, oppure se avete voglia di accedere ai laboratori lì a Udine, la village o qui a Pordenone, i laboratori del CEPO, è possibile concordare un progetto pratico che viene diciamo svolto utilizzando un dispositivo reale. Il dispositivo reale può essere poi controllato all'interno di un progetto di robotica autonoma o attraverso strumenti e librerie di controllo di questo dispositivo ad hoc, quindi che vengono fornite dal produttore del dispositivo, ma sempre con supporto linguaggio Python, oppure attraverso ciò che abbiamo imparato nell'ambiente ROS, perché l'ambiente ROS permette di interfacciarsi con i dispositivi BRAC, per cui diciamo a quel punto è possibile poi lavorare con dispositivi fisici anche tramite usando gli strumenti che abbiamo imparato a conoscere durante la seconda parte del corso. Questo è un aspetto su cui torneremo alla fine, insomma abbiamo bisogno di vedere varie cose prima di riuscire a capire che tipo di progetti noi vi proponiamo, oppure da parte vostra riuscire a proporci, se voi avete da proporci qualche cosa che vi piace in particolare o che volete sviluppare e che sia altinente col costo, possiamo parlarne senz'altro, questa è un po' la modalità con cui poi si svolgerà. L'esame finale scritto vi permette di acquisire un tot di punti in trentesimi che se non ricordo male si aggira intorno ai 18-20, quindi da solo diciamo generalmente non da il punteggio o comunque noi sconsigliamo di passare l'esame soltanto con l'esame finale scritto, piuttosto concordiamo per la parte pratica qualcosa che sia nelle vostre corde, ma l'esame dovrebbe essere la somma delle due parti, anche perché sicuramente la parte progettuale è quella più divertente, generalmente ha poco senso chiudere l'esame senza fare la parte progettuale, generalmente non è mai niente di così pesante o difficile da prendervi più di un paio di settimane non full time di lavoro dovreste riuscire, poi dipende, ripeto, da quanta passione avete per la materia e che tipo di progetto si concorda. Questo per quanto riguarda gli aspetti della logistica e dei contenuti e della gestione del corso, se avete domande rispetto a queste cose qui fatemele pure che io mi dimentico spesso e volentieri a Roma, quindi ditemi pure, intanto io cambio slide. Sempre nella giornata di oggi del Moodle dovreste, vi ho mostrato il Moodle, sì, vi ho mostrato il Moodle prima, quindi dovreste trovare anche un pdf di introduzione a Mobile Robotics dove adesso più o meno cerchiamo di dare qualche indicazione più precisa rispetto a quali sono le classi dei dispositivi che affronteremo nel corso e quali sono gli aspetti della robotica legati a questi dispositivi che approfondiremo. Ok, scusatemi che guardo un attimo l'ora, io generalmente tendo a non fare la pausa in queste lezioni di due ore, siete d'accordo, possiamo andare dritti, abbiamo perso anche un po' di tempo. Possiamo andare dritti fino alle quattro e un quarto, va bene, io di solito inizio alle due e tre quarti più o meno, ok, dobbiamo riuscire generalmente a chiudere per le quattro e un quarto, quattro e venti, va bene, ecco questi gli aspetti che vi racconto tra un attimo sono abbastanza, diciamo, sono tutti più o meno riferiti al DUDE che al SIGGRAD, insomma a SIGGRAD dovete trovare tutto quello che vi serve in quei testi lì e partiamo da una panoramica abbastanza ampia di che cosa significa Mobile Robotics, cioè di che cosa sono i robot mobili, questo perché naturalmente non esistono solo robot mobili, esistono robot che sono fermi, che non hanno possibilità, capacità di movimento o che hanno capacità di movimento molto limitate e in particolare i robot di cui ci occuperemo noi sono quelle piattaforme robotiche che hanno la possibilità, la capacità di muoversi, cioè di cambiare la loro posizione all'interno di un particolare contesto, può essere un contesto all'aperto, outdoor oppure può essere un contesto, un environment al chiuso, un indoor environment e le due cose hanno una serie di implicazioni, si distinguono, si tengono distinte generalmente perché hanno implicazioni abbastanza diverse e in termini di controllabilità degli agenti al contorno, d'accordo, questo è ancora più vero quando poi si deve avere a che fare con i sensori di bordo perché le condizioni in cui devono operare i sensori sono molto diverse se noi consideriamo le condizioni in un contesto all'aperto con diciamo vari agenti non sempre controllabili mentre invece al chiuso spesso si riescono a definire delle condizioni molto più controllate, è più semplice muoversi, quindi faremo riferimento a questi aspetti, a questi due ambiti e i robot, le piattaforme mobili, i robot mobili in generale hanno una qualche forma di guida, cioè si muovono perché sono operati, teleoperati in qualche modo, ad esempio attraverso dei sistemi di guida in radiofrequenza, classico radiocomando, in alcuni casi sono filo guidati, vedremo alcuni esempi di... questo capita spesso nei robot mobili per uso anfibio, quindi sottomarini, in quel caso spesso succede di avere un sistema filo guidato e in altri casi possiamo affidarci a sistemi wifi, quindi a un sistema di controllo senza fili ma che utilizza un sistema di trasmissione non radiofrequenza ma diciamo più simile a un sistema di telecomunicazione, wireless, ecco ma non sempre però i robot mobili hanno bisogno di una, diciamo, guida che dica loro istante per istante il movimento che devono attuare, cioè il tipo di direzione che devono prendere, il tipo di movimento che devono intraprendere e così via, e siamo al terzo punto dove si parla di autonomia e questo infatti è l'aspetto poi che tratteremo un po' più avanti, cioè che cosa significa poter dare a un robot una capacità di autonomia rispetto al movimento in un ambiente complesso, naturalmente non l'autonomia non si esaurisce solo nel movimento, non si esaurisce solo nella capacità di poter accendere o spegnere dei motori o girare, diciamo orientare la prua del robot in una certa direzione, ma deve naturalmente tenere in considerazione una serie di aspetti che servono a decidere come muoversi e che partono dall'analisi dell'ambiente esterno attraverso i sensori e passano per la capacità di prendere decisioni, capacità di prendere decisioni sulla base della percezione dell'ambiente esterno, sulla base della conoscenza della propria posizione in questo ambiente esterno e delle proprie capacità di movimento, della propria dinamica in alcuni casi, e fino a poi a poter, diciamo sulla base di tutte queste conoscenze, poter generare dei segnali di controllo verso i propri attuatori in modo da riuscire a far muovere la piattaforma, così come le proprie logiche di controllo e le logiche di decisione richiedono. Fate attenzione che quando parliamo di robot mobili intendiamo robot che riescono a cambiare, a mutare la posizione del proprio baricentro, della propria base, d'accordo, in maniera svincolata rispetto a vincoli che li tengano legati all'ambiente in cui si muovono. Il caso di sinistra che vedete rappresentato a sinistra, che rappresenta un braccio meccanico, è un esempio di una piattaforma robotica che ha delle capacità di movimento, d'accordo, perché ha anche delle capacità di movimento autonome in molti casi, perché ha la capacità di sentire, nei casi più sofisticati, di percepire l'ambiente. Molto spesso i bracci meccanici robotici come quello schematizzato hanno dei sensori di movimento distribuiti lungo, diciamo, i vari componenti del braccio, hanno delle videocamere posizionati sulla testa, sul gripper, che gli permette di avere un particolare punto, una particolare visione, diciamo, solidale con la punta dell'articolatore, col gripper, con la pinza. Hanno una serie di sensori di rotazione che gli permette di capire come stanno rotando e quindi hanno tutti gli elementi che gli serve a capire qual è la loro posizione all'interno dell'ambiente e come si configura l'ambiente. Se sono dotati poi di una logica, diciamo, di movimento, hanno le capacità di generare impulsi di controllo verso i motori, che gli permette quindi di operare in autonomia, tuttavia non sono mobili nel senso che la loro base è fissa, è ancorata in un certo punto dell'ambiente circostante. Perché tengo a sottolineare questo aspetto? Se è a confrontarlo con invece i robot che hanno una capacità di una mobilità completa all'interno dell'ambiente, come quello rappresentato a destra. Perché ci sono implicazioni importanti dal punto di vista dei sistemi di riferimento implicati nei due scenari. Nel scenario a sinistra il sistema di riferimento è tipicamente uno ed è fisso. Per esempio si può prendere il sistema di riferimento il cui zero è centrato con la base del robot articolatore e tutti i movimenti che questo robot articolatore farà saranno riferiti a questo sistema di riferimento che non varia nel tempo, è vero che in alcuni casi per modellare il movimento si possono poi considerare una serie di sistemi di riferimento solidari con i giunti, per cui a volte vedrete delle piccole terne di assi centrate sui giunti degli articolatori, se ne vedrete non in questo corso in particolare, ma se ne vedrete in futuro. E allora lì ogni giunto ha il suo sistema di riferimento eccetera, ma questo è un aspetto un po' più diciamo microscopico, per quanto riguarda gli aspetti macroscopici senz'altro possiamo fare riferimento ad un unico sistema di riferimento fisso. Cosa che invece cambia quando guardiamo a destra, a destra è raffigurato un wheeled drones, impareremo a conoscerlo perché ha un sistema di guida basato su differential wheel, differential drive scusate si dice, un sistema che prevede di dover controllare le due ruote in maniera individuale, in maniera separata per gestire le rotazioni sull'asse e in questo caso, comunque in generale nel caso di una piattaforma robotica che si muove all'interno di un ambiente più o meno complesso, il sistema di riferimento come vedete non diventa univoco perché noi possiamo avere un sistema di riferimento solidale con la stanza o con l'ambiente in cui si sta muovendo il drone, il robot e allora in quel caso quel sistema di riferimento è fisso e immutabile nel tempo, non ruota e non trasla, mentre invece è possibile definire dei sistemi, un sistema di riferimento invece solidale con il drone, in questo caso raffigurato nel piano bidimensionale che taglia la piattaforma mobile orizzontalmente e se volete potete immaginare un terzo asse che viene verso di voi, l'asse z, e quel sistema di riferimento è un sistema di riferimento invece solidale che cambia nel tempo man mano che il drone si muove, sia in termini di rotazione sia in termini di traslazione, quindi voi immaginate il movimento di questo oggetto al passare del tempo e potete così immaginare come il sistema di riferimento trasli e ruoti rispetto a un sistema di riferimento invece fisso che è quello della stanza o dell'ambiente autore in cui stiamo muovendo, questi due sistemi di riferimento diciamo cambiano in continuazione l'uno rispetto all'altro, scusate, le posizioni, le cifre cambiano rispetto all'altro in continuazione. Il sistema di riferimento solidale con il robot è generalmente chiamato local reference system, sistema di riferimento locale e vedremo che tutta una serie di elaborazioni sono fatte con riferimento a questo sistema locale perché molto spesso è necessario, per esempio, per quanto riguarda la parte sensoristica, la parte diciamo di positioning individuale, cioè della capacità di capire da parte del robot in che punto si trova con riferimento allo spazio circostante e così via, tante cose devono essere riferite al sistema di riferimento locale. Mentre invece altre informazioni, scusate ho detto una cosa che mi corrego subito, l'informazione del positioning per esempio può essere riferita sia al sistema di riferimento locale, d'accordo, ma molto più spesso è riferita al sistema di riferimento globale, sistema di riferimento solidale con la stanza o con l'ambiente esterno e così è chiamato sistema di riferimento globale. Allora la posizione del robot stimata per esempio attraverso dei sensori oppure calcolata attraverso la cinematica e il movimento, quella posizione tipicamente è riferita al sistema di riferimento globale. Mentre al sistema di riferimento locale possono essere riferiti informazioni del campo visivo catturato dalla videocamera, per esempio alle sorgenti acustiche rilevate dai sensori acustici, ai radiofari rilevati o ai sensori di prossimità rilevati con altri sistemi e così via, d'accordo. Ecco, fatta questa veloce introduzione, come vi anticipavo prima, noi non considereremo sistemi robotici come quello di sinistra, cioè sistemi robotici che si muovono, che articolano ma che non si muovono al pari di un sistema su ruota o di un sistema subaqueo sottomarino oppure di un sistema aereo, d'accordo. Noi considereremo principalmente robot del tipo di destra e anche all'interno di questa grande classe di robot che hanno capacità di movimento all'interno dell'ambiente, anche all'interno di questa vasta famiglia faremo una selezione. In particolare questa slide che permette di dare uno sguardo sulle diverse applicazioni in cui oggi sono coinvolti i droni, mi permette di segnalarvi già subito quali sono i particolari dispositivi che andremo a vedere meno e quali quelli che andremo a vedere più in maniera approfondita. In particolare riferiamoci in questo momento a un contesto indoor, cioè un contesto controllato, un ambiente chiuso, una stanza, proviamo a discutere diciamo i vari esempi uno per uno, partendo da quello più in alto, dove vedete alcune possibili realizzazioni di robot impiegabili nel contesto, in un contesto domestico oppure in un contesto ospedaliero, in un contesto in cui il robot svolga funzioni di assistenza o di ausilio ai compiti diciamo domestici, possono essere gestione degli ambienti domestici per la pulizia, possono essere applicazioni di assistenza agli anziani oppure assistenza a pazienti in ambito ospedaliero, possono essere aspetti legati all'entertainment oppure al cosiddetto edutainment, cioè a quelle applicazioni che prevedono di diciamo trasmettere delle conoscenze, in alto a destra avete un paio, in alto a destra con in particolare quel piccolo robot pet robot chiamiamolo, cioè robot giocattolo assimilabile a un piccolo pupazzo diciamo così come sono abituati ad avere i bambini, oppure quello di destra è un robot un po' più sofisticato ma anche esso potrebbe essere usato in ambiti diciamo di didattica di età infantile, ecco questi sono esempi diciamo in cui le piattaforme robotiche oggi rispetto a qualche anno fa sono senz'altro molto più molto più presenti e soprattutto le loro caratteristiche, le loro capacità sono molto più sofisticate di qualche di una decina d'anni fa. Adesso per entrare un po' nello specifico degli esempi lì in alto vedete un robot rumba utilizzato per la pulizia degli ambienti domestici, qualcosa di simile è disponibile oggi a costi sicuramente accessibili a tutti per la pulizia, per la gestione degli ambienti verdi, degli spazi verdi, qui siamo in indoor ma mi ricollego solo a piccoli spazi come il giardino di casa eccetera oppure la pulizia di vasche per esempio è molto comune trovare questo tipo di oggetti nelle piscine per la pulizia quotidiana. I piccoli robot che vedete a destra, i pet robots, questo è un esempio oggi disponibile diciamo ma poco sofisticato ma il capostipite di quel tipo di robot è stato per chi forse qualcuno lo conosce AIBO, AIBO era un piccolo cagnolino, un piccolo robot con le fattezze di un cagnolino che aveva delle caratteristiche molto interessanti dal punto di vista sia del comportamento sia della sensoristica cioè era dotato di sensori visivi e uditivi che permettevano di interagire con i bambini, era in particolare usato in contesto in contesto prescolare o in età scolare e soprattutto nell'ambito della ricerca robotica, ricerca che coinvolgeva naturalmente l'interazione con soggetti piccoli diciamo di età piccoli soggetti di età scolare. AIBO permetteva poi, aveva dei movimenti molto avanzati per l'epoca, permetteva di esprimere diciamo alcune caratteristiche dell'emotività, riusciva ad essere emotivo e infatti i bambini riuscivano a comunicare e ad interagire con lui in maniera molto efficace. Il robot che vedete invece più a destra in alto si chiama Nao, noi abbiamo un esemplare di quel robot, una copia di Nao, scusate un dispositivo Nao qui a Pordenone e viene utilizzato nel corso per esempio di GMT del DiSocial Robotics e anche questo è un robot estremamente avanzato dal punto di vista della capacità di interazioni sociali verso le persone. Un po' meno quelli che vedete in basso, questi sono robot un po' meno diciamo sofisticati dal punto di vista dell'interazione, però insomma intuite le potenzialità. Oggi questo tipo di robot si incontrano per esempio comunemente nei ristoranti e il motivo è che non tanto per l'efficienza che hanno, che è ancora relativamente bassa, ma naturalmente per l'attrattività che hanno. Insomma questi sono in grado di venire al tavolo, di prendere ordinazioni e di poi arrivare con un vassoio, con la vostra ordinazione e chiedervi di prenderla e quindi sostituire quasi in toto un inserviente. Così gli esempi più in basso rappresentano esempi di robot che permettono di portare assistenza ad un malato ad esempio, quindi portare acqua o alcune diciamo medicine o farmaci che il malato può gestirsi in autonomia e con l'aiuto di questo dispositivo insomma riesce a gestirla in maniera più efficiente. L'ospedale, scusate, riesce a gestire in maniera più efficiente il controllo del reparto. Altri esempi di applicazioni in sorveglianza indoor, qui è abbastanza ovvio che questo sensore, questo robot in base a destra si muove attraverso un sistema di cingoli e permette di acquisire dati. Quindi tipicamente il compito di uno di questi dispositivi è quello di esplorare continuamente e secondo dei pattern predefiniti più o meno regolari, a seconda dell'applicazione, un ambiente sotto sorveglianza e qui tipicamente alcune capacità di azione sono in sizione nel robot, quindi è in grado di prendere alcune decisioni da solo, oppure se è in comunicazione con una sala controllo a cui spedisce o invia, cui trasmette immagini e audio, la sala controllo è in grado di teleoperare il robot a distanza attraverso dei comandi, sia gestiti con algoritmi, sia gestiti da un operatore in sala di controllo. Ancora ricerca e sviluppo, quello che vedete in basso nel contesto di ricerca e sviluppo è un esempio di uno dei primi wheeled robots usati in contesto accademico e industriale, è un Pioneer P3 se non ricordo male, di cui noi abbiamo un esemplare qui a Pordenone, nei laboratori qui a Pordenone a Cepo, ed era uno dei primi semplici robot che permetteva di controllare attraverso dei sensori disposti sulla piattaforma che vedete in alto e attraverso dei sistemi in calcolo posati sulla sua base diciamo orizzontale in alto appoggio, era in grado di muoversi e prendere semplici decisioni sulla base dei sensori e sulla base degli algoritmi programmati all'interno del sistema di calcolo a bordo. Ancora applicazioni di tipo indoor, li trovate per esempio nella logistica, la logistica oggi è uno degli ambiti industriali in cui c'è stata una piccola rivoluzione negli ultimi anni per quanto riguarda l'utilizzo di robot mobili all'interno dei locali di stoccaggio e qui naturalmente c'è da notare che cosa se provate a immaginare il lavoro che deve fare un robot in questo ambiente, è un ambiente relativamente comodo per un dispositivo del genere perché è un ambiente molto diciamo che non cambia dal punto di vista della configurazione degli spazi, non cambia dinamicamente, gli scaffali sono fermi, non si muovono e forse l'unica cosa che cambia dinamicamente nel tempo è la quantità di merce, quindi la disposizione della quantità di merce e la disposizione degli oggetti, delle scale, degli scatoloni sui scaffali ma è una cosa relativamente controllabile non soltanto attraverso tecniche di machine vision ma anche attraverso sensori, quindi se gli scaffali sono dotati di sensori non è difficile diciamo avere un'idea sempre aggiornata e abbastanza precisa di qual è lo stato dell'ambiente in un certo istante, quindi il robot principalmente muoversi in una mappa precostituita che non cambia sapendo a priori quale è la disposizione degli oggetti a cui diciamo giungere e quali sono quelli da prendere e così via, invece leggermente più complesso se volete l'indoor farming, l'indoor farming è un ambito di applicazione che anche qui sta diventando estremamente interessante e diffuso oggi in cui i robot sono utilizzati per gestire la cura di piantagioni sia indoor che in condizioni outdoor, vedremo nella slide successiva, ma comunque per gestire una serie di aspetti che hanno a che fare con la cura della vegetazione. Datemi ancora solo cinque minuti scusate oggi abbiamo perso un po' di tempo in avvio, cinque minuti così chiudo questo discorso poi mi fermo perché volevo appunto raccontarvi come oggi questi dispositivi mobili sono in grado di analizzare e di gestire la distribuzione sia di acqua che di sostanze curative antiparassitari eccetera per le piante sulla base di informazioni gestite sempre attraverso sensori ma elaborate attraverso anche sistemi di imaging e di computer vision molto sofisticati con le quali si può ad esempio riuscire a capire in maniera automatica quali sono le piante che hanno bisogno di cure e di quali cure o di acqua e di quanta acqua nell'ambiente indoor, quali piante ad esempio hanno bisogno di essere potate oppure di effettuare delle raccolte di frutta in certe condizioni e così via e questo stesso aspetto si incontra oggi nell'ambito dell'outdoor farming è chiamato cioè non esiste un outdoor farming o forse sì ma ci si riferisce più a un ambito cosiddetto di agricoltura di precisione precision agriculture in cui diciamo i robot i droni sono utilizzati per lo spargimento di sostanze dall'alto per l'individuazione di zone delle colture che hanno problemi oppure che hanno un certo grado di maturazione che hanno bisogno meno di altre di acqua o di sostanze antiparassitarie o curative eccetera. La precision agriculture è sicuramente uno degli ambiti in cui i droni aerei oggi sono molto impiegati unitamente appunto alla computer vision come vi accennavo prima. Andando girando diciamo anche qui da sinistra verso destra vedete un esempio di underwater drone, un drone sommergibile che viene tipicamente impiegato per esplorazioni marine, esplorazione di fondali, esplorazione di ricerca di relitti o di oggetti sul fondale, direi anche spesso è utilizzato in operazioni di manutenzione nelle dorsali per esempio quando si tratta di fare interventi di operare interventi di manutenzione ad altissima profondità su dorsali di reti di telecomunicazioni oppure anche su pipeline per la distribuzione di gas o di combustibile e così via. Sono sistemi spesso filodiretti, hanno un collegamento via cavo per motivi di autonomia e di propagazione del segnale radio ma da lunghe distanze, quindi tipicamente sono sono diciamo agganciati tramite un cavo ad una nave di controllo, cosa che invece è più complicato per i droni aerei per lo stesso tipo di applicazioni, ma i droni aerei come vedremo non è così facile avere un controllo filodiretto. E nell'ambito applicazioni in ambito aerospaziale solo pochi anni fa nel 2021-22, non ricordo, insomma abbiamo assistito all'atterraggio di un dispositivo mobile come quello raffigurato in figura, un cosiddetto rover, sulla superficie di Marte e in quell'occasione ci hanno stupito, se vi ricordate, le immagini ad altissima risoluzione, la qualità dell'audio che riprendeva non tanto le caratteristiche acustiche di Marte, perché voi sapete che non c'è, essendoci un'atmosfera così particolare, non c'è neanche tanto rumore, ma insomma tutta una serie di rumori dovuti alle vibrazioni in fase di atterraggio sono stati rilevati. Dopodiché sono droni spesso eterodiretti, quindi controllati nel movimento, eccetera, dalle stazioni sulla Terra, però hanno già oggi capacità di autonomia per piccoli operazioni di prelievo dei campioni. Ambito militare, droni ambito militare, sapete tutti quale sono le applicazioni e gli impieghi da vari anni a questa parte, sia di difesa sia di offesa e nei movimento di un drone ad ala fissa, cosiddetto, un drone aereo ad ala fissa in alto a sinistra rispetto a un drone aereo ad ala adelica in alto centro, centro alto. Questi droni hanno capacità di movimento molto diverse rispetto a quelli ad ala fissa, sono molto più controllabili, hanno naturalmente velocità di spostamento inferiori e anche autonomia inferiore generalmente, ma hanno capacità di controllo molto più precisa, possono lavorare in condizioni di hovering, cosa che non può fare un drone ad ala fissa e questo gli permette di svolgere funzioni molto diverse da quelle di un drone ad ala fissa. Per esempio possono posizionarsi in maniera stazionaria su un target, osservarlo da vicino, avvicinarsi, mantenere distanza costante, operare a quote più basse, operare anche in condizioni di occlusione rispetto al target osservato, ad esempio nascondersi dietro alcune e alcune volte sono talmente poco rumorosi, non tutti, ma alcuni modelli che possono arrivare indisturbati molto vicino a un target prima di essere rilevati appunto acusticamente. E infine nale, ambienti dove un soccorritore per esempio fa fatica ad arrivare, questo sia dal punto, sia dall'aria che da terra. Qui vedete in particolare il search and rescue attraverso un drone dotato di cingoli per spostamento a terra, ma la stessa cosa può essere immaginata attraverso l'utilizzo di droni aerei per arrivare in zone dove, per esempio, colpite da catastrofi naturali, terremoti, uragani o altro, dove non è facile per un soccorritore umano arrivare in prossimità, ad osservare così da vicino alcuni particolari punti dell'ambiente da perlustrare e addirittura in certi casi permettere di instaurare una comunicazione con, per esempio, persone intrappolate in modo da poter poi decidere e capire come agire, come intervenire. Direi che abbiamo più o meno chiarito il quadro, se avete domande ditemi pure a questo punto qualcosa che vi ha incuriosito o qualcosa che ho dimenticato di trattare e volentieri vi... Ecco, allora, intanto che voi ragionate io spengo la registrazione. Come al solito non è un'operazione semplice, vediamo un attimo. Ecco qua, tolto l. """},
]


text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to("cuda")
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]